In [ ]:
import pygame
import numpy as np
import math,os
from swerve_drive import SwerveDrive
from PIDController import PIDController
from TrajectoryParser import AutoParser
from swerve_environment import SwerveEnvironment
from config import *
from Genetic_Algo import GeneticAlgorithm
# Configure matplotlib for Google Colab (prevents flashing)
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()  # Turn on interactive mode

print("✓ Matplotlib configured for smooth updates in Google Colab")

In [ ]:
# Create environment
start_point = (4,4,0)
env_HW = (8.21, 16.54)  # FRC field dimensions in meters (height, width)
env = SwerveEnvironment(width=1000, height=800, dt=0.02,HW = env_HW, start_point = start_point)


In [ ]:

auto_parser = AutoParser(
    r"c:\Users\chris.reckner\Documents\pythontests\Machine Learning\pathplanner\autos\Wonky 3.auto",
    paths_directory=r"c:\Users\chris.reckner\Documents\pythontests\Machine Learning\pathplanner\paths"
)

print(f"\nPath names in auto: {auto_parser.get_path_names()}")

# Get summary
summary = auto_parser.get_trajectory_summary()
print(f"\nAuto Summary:")
print(f"  Number of paths: {summary['num_paths']}")
print(f"  Total duration: {summary['duration']:.2f} seconds")
print(f"  Total distance: {summary['total_distance']:.2f} meters")
print(f"  Start position: ({summary['start_position'][0]:.2f}, {summary['start_position'][1]:.2f})")
print(f"  End position: ({summary['end_position'][0]:.2f}, {summary['end_position'][1]:.2f})")

# Generate combined trajectory
combined_trajectory = auto_parser.generate_combined_trajectory(dt=0.25)



In [ ]:

# Global variables to hold figure, axes, and image plot for continuous updates
_global_fig = None
_global_axs = None
_global_display_handle = None
_global_img_plot = None

def _init_plot_for_eval():
    """Initializes the matplotlib figure and axes for live rendering, only once."""
    global _global_fig, _global_axs, _global_display_handle, _global_img_plot
    if _global_fig is None:
        # Create figure and a GridSpec for layout
        _global_fig = plt.figure(figsize=(18, 6), constrained_layout=True)
        gs = _global_fig.add_gridspec(2, 3) # 2 rows, 3 columns

        # Define subplots based on the grid
        ax_cart = _global_fig.add_subplot(gs[:, 0]) # CartPole visualization (takes all rows in first column)
        ax_fitness = _global_fig.add_subplot(gs[0, 1:]) # Fitness plot (takes top row, second and third columns)
        ax_pop = _global_fig.add_subplot(gs[1, 1:]) # Population plot (takes bottom row, second and third columns)

        _global_axs = {'cart': ax_cart, 'fitness': ax_fitness, 'pop': ax_pop}

        # Get a display handle for updating the figure without clearing output
        _global_display_handle = display(_global_fig, display_id=True)
        plt.close(_global_fig) # Prevent initial display of empty figure below the handle
        _global_img_plot = None # Initialize img_plot as None
    return _global_fig, _global_axs, _global_display_handle

In [ ]:
def render_frame(frame,fig, axs, display_handle,ax_cart, ax_fitness, ax_pop,
                                 ga_instance, current_gen, current_individual,error=None,time=None,):
    frame = env.render()

    # Clear existing axes content before redrawing
    ax_cart.clear()
    ax_fitness.clear()
    ax_pop.clear()

    # CartPole visualization
    global _global_img_plot # Access the global image plot handle
    if _global_img_plot is None or _global_img_plot.axes != ax_cart:
        _global_img_plot = ax_cart.imshow(frame)
    else:
        _global_img_plot.set_data(frame)
    ax_cart.axis('off')
    ax_cart.set_title(f'Gen {current_gen}/{ga_instance.generations} | Individual {current_individual}/{ga_instance.population_size}\n' +
                         f'Time: {time}\n' +
                        f'Reward: {error:.1f}',
                        fontsize=12, fontweight='bold')

    # Top Right: Best and Average Fitness Over Generations
    if ga_instance.best_fitness_history:
        generations_hist = range(1, len(ga_instance.best_fitness_history) + 1)
        ax_fitness.plot(generations_hist, ga_instance.best_fitness_history,
                        'g-o', linewidth=2, markersize=6, label='Best Fitness')
        ax_fitness.plot(generations_hist, ga_instance.avg_fitness_history,
                        'b-s', linewidth=2, markersize=4, alpha=0.7, label='Avg Fitness')
        ax_fitness.axvline(x=current_gen, color='red', linestyle='--',
                            linewidth=2, label='Current Gen', alpha=0.7)
        ax_fitness.set_xlabel('Generation', fontsize=11)
        ax_fitness.set_ylabel('Fitness', fontsize=11)
        ax_fitness.set_title('Evolution Progress', fontsize=12, fontweight='bold')
        ax_fitness.legend(loc='best')
        ax_fitness.grid(True, alpha=0.3)
    else:
        ax_fitness.text(0.5, 0.5, 'Waiting for generation data...',
                        ha='center', va='center', fontsize=12, transform=ax_fitness.transAxes)
        ax_fitness.set_xlim(0, 1)
        ax_fitness.set_ylim(0, 1)

    # Bottom Right: Current Generation Population Performance
    if ga_instance.current_generation_fitness is not None:
        individuals_pop = range(1, len(ga_instance.current_generation_fitness) + 1)
        colors = ['red' if i == current_individual else 'skyblue'
                    for i in individuals_pop]
        bars = ax_pop.bar(individuals_pop, ga_instance.current_generation_fitness,
                            color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

        # Highlight current individual
        if current_individual is not None and current_individual <= len(bars):
            bars[current_individual-1].set_linewidth(3)
            bars[current_individual-1].set_edgecolor('darkred')

        ax_pop.axhline(y=np.mean(ga_instance.current_generation_fitness),
                        color='blue', linestyle='--', linewidth=2,
                        label=f'Gen Avg: {np.mean(ga_instance.current_generation_fitness):.1f}', alpha=0.7)
        ax_pop.axhline(y=np.max(ga_instance.current_generation_fitness),
                        color='green', linestyle='--', linewidth=2,
                        label=f'Gen Best: {np.max(ga_instance.current_generation_fitness):.1f}', alpha=0.7)

        ax_pop.set_xlabel('Individual', fontsize=11)
        ax_pop.set_ylabel('Fitness', fontsize=11)
        ax_pop.set_title(f'Generation {current_gen} Population Performance',
                        fontsize=12, fontweight='bold')
        ax_pop.legend(loc='best')
        ax_pop.grid(True, alpha=0.3, axis='y')
    else:
        ax_pop.text(0.5, 0.5, 'Evaluating population...',
                    ha='center', va='center', fontsize=12, transform=ax_pop.transAxes)
        ax_pop.set_xlim(0, 1)
        ax_pop.set_ylim(0, 1)

    # Update the displayed figure
    display_handle.update(fig)


In [ ]:
def evaluate_individual(individual, render=True,
                        ga_instance=None, current_gen=None, current_individual=None, episodes=1,render_steps =5):
    # Create PID controllers (assuming they are defined elsewhere)
    accumulated_error = 0
    if render:
        fig, axs, display_handle = _init_plot_for_eval()
        ax_cart = axs['cart']
        ax_fitness = axs['fitness']
        ax_pop = axs['pop']
    for ep in range(episodes):
        trans_pidx = PIDController(kp=individual[0], ki=individual[1], kd=individual[2])
        trans_pidy = PIDController(kp=individual[0], ki=individual[1], kd=individual[2])
        angular_pid = PIDController(kp=individual[3], ki=individual[4], kd=individual[5])
        accumulated_error_episode = 0.0
        time = 0.0
        trajectory_data = []
        env.reset(4,4,0)
        trajectory = combined_trajectory
        step = 0
        for waypoint in trajectory:
            # Get current state
            while time<waypoint['time']:
                time += env.dt
                step += 1
                state = env.get_state()
                current_x = state['x']
                current_y = state['y']
                current_theta = state['theta']
                
                # Calculate position errors
                error_x = waypoint['x'] - current_x
                error_y = waypoint['y'] - current_y
                trans_error = math.sqrt(error_x**2 + error_y**2)
                
                # Calculate angular error (shortest angle)
                error_theta = waypoint['theta'] - current_theta
                error_theta = math.atan2(math.sin(error_theta), math.cos(error_theta))
                
                # Convert translational error to robot frame
                cos_theta = math.cos(current_theta)
                sin_theta = math.sin(current_theta)
               
                
                # PID control for translational motion
                vx_correction = trans_pidx.update(error_x,env.dt)
                vy_correction = trans_pidy.update(error_y,env.dt)
                # print(vx_correction,vy_correction, "|",error_x,error_y)
                # PID control for angular motion
                omega_correction = angular_pid.update(error_theta,env.dt)
                
                # Combine feedforward and feedback
                vx_cmd = vx_correction
                vy_cmd = vy_correction
                omega_cmd = omega_correction
                
                # Step simulation
                step_state = env.step(vx_cmd, vy_cmd, omega_cmd)
                trajectory_data.append({
                    'state': step_state,
                    'target': (waypoint['x'], waypoint['y'], waypoint['theta']),
                    'error': (trans_error, error_theta),
                    'command': (vx_cmd, vy_cmd, omega_cmd)
                })
                accumulated_error_episode = trajectory_data[-1]['error'][0] + trajectory_data[-1]['error'][1]
                # Render if enabled
                if render and step % render_steps == 0:
                    frame = env.render()
                    if not env.handle_events():
                        return False, trajectory_data
                    env.clock.tick(int(1.0 / env.dt))
                    render_frame(frame,fig, axs, display_handle,ax_cart, ax_fitness, ax_pop,
                                 ga_instance, current_gen, current_individual,error=accumulated_error_episode,time=time)
        
        # Accumulate error for this episode
        accumulated_error += accumulated_error_episode
            
    return accumulated_error / episodes

In [ ]:
ga = GeneticAlgorithm(
    population_size=POPULATION_SIZE,
    generations=GENERATIONS,
    visualize_best=VISUALIZE_BEST,
    evaluate_individual=evaluate_individual,
    parameter_ranges=PID_RANGES)

In [ ]:
#========================================
# No Changes need to be made in this cell
#========================================
# Time to Run the Genetic Algorithm

# This looks intimidating, but it's just setting up and running the 
# GA with the parameters you've defined above. Most of the code is for plotting
# and displaying results nicely.

print("\n" + "="*70)
print("STARTING GENETIC ALGORITHM")
print("="*70)


print(f"\nSearch Configuration:")
print(f"  Population: {ga.population_size} individuals")
print(f"  Generations: {ga.generations}")
print(f"  Parameter ranges: Kp{ga.kp_range}, Ki{ga.ki_range}, Kd{ga.kd_range}")
print(f"  Mutation: rate={MUTATION_RATE}, effect={MUTATION_EFFECT}")
print(f"  Crossover rate: {CROSSOVER_RATE}")
print(f"  Elitism: {ELITISM}")
print("="*70 + "\n")

best_pid, best_fitness = ga.evolve()

print("\n" + "=" * 60)
print("FINAL RESULTS")
print("=" * 60)
print(f"Best PID Parameters: Kp={best_pid[0]:.3f}, Ki={best_pid[1]:.3f}, Kd={best_pid[2]:.3f}")
print(f"Best Fitness (Average Reward): {best_fitness:.2f}")

# Plot the evolution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(ga.best_fitness_history, 'g-o', linewidth=2, markersize=8, label='Best Fitness')
plt.plot(ga.avg_fitness_history, 'b-s', linewidth=2, markersize=6, label='Average Fitness')
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Fitness (Average Reward)', fontsize=12)
plt.title('Genetic Algorithm Evolution', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
improvement = np.array(ga.best_fitness_history) - ga.best_fitness_history[0]
plt.bar(range(1, len(improvement)+1), improvement, color='green', alpha=0.6, edgecolor='black')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Improvement from Gen 1', fontsize=12)
plt.title('Fitness Improvement Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()